In [1]:
import json
import pandas as pd
search_engine = "brave"
model_name = 'mixtral'

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)

def load_diff(model_name, diff_version):
    with open(f"output/diff_{model_name}_v{diff_version}.json", "r") as f:
        return json.load(f)

def save_excel(file_name, claims, labels, preds=None):

    if preds:
        df = pd.DataFrame(columns=["claim", "label", "prediction"])
        data ={'claim': claims, 'label':labels, 'prediction':preds}
        for i, row in enumerate(data['claim']):
            df.loc[i] = [data['claim'][i], data['label'][i], data['prediction'][i]]
    else:
        df = pd.DataFrame(columns=["claim", "label"])
        data ={'claim': claims, 'label':labels}
        for i, row in enumerate(data['claim']):
            df.loc[i] = [data['claim'][i], data['label'][i]]
    df.to_excel(f'{file_name}.xlsx')


In [2]:
wrong_claims_dir = "../test_metric_single_llm/with_mixtral_info/brave/data1-split=0.5:9.5-ratio=1.0/lora-r=2/openchat-3.5-0106-lr=2e-4.txt"
with open(wrong_claims_dir) as f:
    wrong_claims = f.readlines()[2:]

wrong_labels = [s[s.find("Label: ") + len("Label: "): s.find(". Pred")].strip() for s in wrong_claims]
wrong_preds = [s[s.find("Pred: ") + len("Pred: "):s.find(". Claim")].strip() for s in wrong_claims]
wrong_claims = [s[s.find("Claim: ") + len("Claim: "):].strip() for s in wrong_claims]

len(wrong_claims)


495

模型分类错误的样例中不包含在先验知识与label不符的样例中的情况：如果这些样例均是label本身就正确，那么“先验知识与label不符”的策略可行

In [3]:
diff_version = 1
wrong_claims_diff = []
data_diff = load_diff("llama3", diff_version)
cnt = 0

check_claim_list = []
check_preds_list = []
check_labels_list = []

for item in data_diff:
    if item["match"].lower() == "no":
        wrong_claims_diff.append(item["claim"])


for i, claim in enumerate(wrong_claims):
    if claim not in wrong_claims_diff: # 检查不在diff中的是不是都是对的
        check_claim_list.append(claim)
        check_preds_list.append(wrong_preds[i])
        check_labels_list.append(wrong_labels[i])

# save_excel("check_claim", check_claim_list, check_labels_list, check_preds_list)

先验知识与label不符的样例

In [5]:
diff_version = 1
wrong_claims_diff = []
labels_diff = []
data_diff = load_diff("llama3", diff_version)

for item in data_diff:
    if item["match"].lower() == "no":
        wrong_claims_diff.append(item["claim"])
        label = int(item["label"])
        if label == 0:
            label = "FALSE"
        elif label == 2:
            label = "TRUE"
        else:
            raise Exception()
        labels_diff.append(label)

save_excel("diff_llama3", wrong_claims_diff, labels_diff)

In [ ]:
websites = [
    'politifact.com/factchecks', 'snopes.com/fact-check', 'factcheck.afp', 'africacheck', 
    'polygraph.info', 
    # 'reuters',  # 
    # 'washingtonpost', # 
    # 'google fact check', 
    'poynter',

    "factcheck"
]

pending_ids = []
pending_labels = []
pending_preds = []

def get_ans(result):
    fact_check_list = ["fact check", "factcheck", "fact-check"]
    for i in fact_check_list:
        if i in result["title"].lower():
            return True

    else:
        for site in websites:
            if site in result["url"].lower():
                return True
    return False

for c, claim in enumerate(wrong_claims):
    for i in range(len(data_search)):
        if claim.strip() in data_search[i]["claim"].strip():
            results = data_search[i][f"{search_engine}_search_results"]["web"]["results"]
            ok = False
            for result in results:
                if get_ans(result):
                    ok = True
                    break
            if not ok:
                pending_ids.append(i)
                pending_labels.append(wrong_labels[c])
                pending_preds.append(wrong_preds[c])
            break


In [ ]:
pending_claims = [data_search[i]["claim"] for i in pending_ids]
len(pending_claims)


299

In [ ]:
import pandas as pd

#创建一个空的 DataFrame
df = pd.DataFrame(columns=["claim", "label", "prediction"])
#添加数据
data ={'claim': pending_claims, 'label':pending_labels, 'prediction':pending_preds}

for i, row in enumerate(data['claim']):
    df.loc[i] = [data['claim'][i], data['label'][i], data['prediction'][i]]

#输出 DataFrame
print(df)
df.to_excel('excel_output.xlsx')
            

                                                 claim  label prediction
0    Traffic images in Madrid for the start of Easter.  FALSE       TRUE
1    A photograph shows the damage to a person's lu...   TRUE      FALSE
2    There will be several thousand dollars in paym...   TRUE      FALSE
3    One of the things we fixed -- if you look at t...  FALSE       TRUE
4    CDC: COVID-19 Can Spread 13 Feet, Travel on Shoes   TRUE      FALSE
..                                                 ...    ...        ...
294  We inherited an obsolete deal [for testing] an...  FALSE       TRUE
295   States Tighten Restrictions as Coronavirus Rages   TRUE      FALSE
296                                   A headline reads  FALSE       TRUE
297  Missouri Family Reports 8 COVID-19 Cases, 3 De...   TRUE      FALSE
298  Hard-Hit Navajo Nation Reports Lower Case Numbers   TRUE      FALSE

[299 rows x 3 columns]


In [ ]:
s = "Information 1, published in 2022, states that people with heart defects are at a greater risk for severe COVID-19, particularly if they have another underlying medical condition such as heart failure, pulmonary hypertension, Down syndrome, diabetes, or obesity. Information 2, published in 2022, discusses the potential for heart problems after COVID-19, including type 2 heart attacks caused by increased stress on the heart. Information 3, published in None, states that COVID-19 may increase the risk of a heart attack or stroke, particularly in people with pre-existing heart conditions such as heart failure, coronary artery disease, and overweight/obesity. Information 4, published in 2013, states that people with mental health and mood disorders, including schizophrenia, are at a higher risk for hospitalization and death from COVID-19 due to pre-existing conditions such as diabetes and heart problems. Information 5, published in None, states that people with pre-existing heart disease who become ill with COVID-19 may suffer a heart attack or develop congestive heart failure due to increased demands on the heart and low oxygen levels.\n\nThe claim, published in 2020, states that heart defects do not increase the risk of severe COVID. However, the information provided in all the other sources contradicts this claim. Information 1, 2, 3, 4 and 5 all suggest that people with heart defects or pre-existing heart conditions are at a greater risk for severe COVID-19 or heart problems after COVID-19. Therefore, the claim is false.\n\nIt is important to note that the information provided here is based on the publication dates of the sources and the information available at that time. The understanding of the relationship between heart defects and COVID-19 may have evolved since the claim was published in 2020."

print(s)


Information 1, published in 2022, states that people with heart defects are at a greater risk for severe COVID-19, particularly if they have another underlying medical condition such as heart failure, pulmonary hypertension, Down syndrome, diabetes, or obesity. Information 2, published in 2022, discusses the potential for heart problems after COVID-19, including type 2 heart attacks caused by increased stress on the heart. Information 3, published in None, states that COVID-19 may increase the risk of a heart attack or stroke, particularly in people with pre-existing heart conditions such as heart failure, coronary artery disease, and overweight/obesity. Information 4, published in 2013, states that people with mental health and mood disorders, including schizophrenia, are at a higher risk for hospitalization and death from COVID-19 due to pre-existing conditions such as diabetes and heart problems. Information 5, published in None, states that people with pre-existing heart disease wh